# Multivariate Linear Regression

In this assignment, you will implement multivariate linear regression to predict the price of houses based on its characteristics.

Suppose you are selling your house and you want to know what a good market price would be. One way to do this is to first collect information on recent houses sold and make a model of housing prices. The file houses.csv contains a training set of housing prices in the city of Valladolid. The first column is the size of the house (in square feet), the second column is the number of bedrooms and the third column is the price of the house. Dataset is like below:

| Size of the house (in square feet) | Number of bedrooms | Price of the house |
|------------------------------------|--------------------|--------------------|
| 2104                               | 3                  | 399900             |
| 1600                               | 3                  | 329900             |
| 2400                               | 3                  | 369000             |


You should fit a multivariate linear regression model using size and bedrooms as variables to predict the price:
* Plot scatterplot of the two variables against price
* Use Scikit-learn to fit the model
* Compute the parameters with the normal equations using np.linalg.inv()
* Compute the parameters with the pseudo-inverse using np.linalg.pinv()
* Plot the fitted surface obtained with any of the previous methods

After that, you should implement the gradient descent algorithm to train a multivariate linear regression model. Try to fit the model with gradient descent and describe the results.

Note that you should standardize the values of the input variables, substracting the mean and dividing by the standard deviation.

For batch gradient descent you should consider the following:

* Define a variable for the learning rate $\alpha$ and try with a default value of 0.01.
* Define a maximum number of iterations with 5000 as default.
* Initialize $\boldsymbol{\theta}$ with random values or zeros.
* Plot J as a function of time (iterations).

Finally, you should implement stochastic and mini-batch gradient descent and compare the parameters obtained with all methods.
- Scikit-learn
- Normal equations
- Pseudo-inverse
- Gradient descent (batch)
- Gradient descent (stochastic)
- Gradient descent (mini-batch)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# Suprimir notación científica en arrays de NumPy para mejor legibilidad
np.set_printoptions(suppress=True)

In [ ]:
# Cargar el dataset de "houses.csv"; se asignan nombres de columna manualmente
# porque el CSV no tiene encabezado
houses = pd.read_csv("houses.csv", names=["size", "rooms", "price"])
houses.head()

In [ ]:
# Separar los features X y la variable objetivo y
# X contiene 'size' y 'rooms'; Y contiene 'price'
X_data = np.array(houses[['size', 'rooms']])       # Matriz de features (n_samples x 2)
y_data = np.array(houses['price']).reshape(-1, 1)  # Vector columna de precios (n_samples x 1) donde reshape cambia la dimensión a una matriz de una sola columna sin modificar los datos
X_data.shape, y_data.shape

### Standardize data

Sometimes can be useful to standardize the input variables to help with convergence in gradient descent. Note that this may not be necessary for sklearn, normal equations and SVD, but done here just to compare results with gradient descent method.

In [ ]:
# Instanciar el escalador y ajustar-transformar las features
# StandardScaler: z = (x - mean) / std
scaler = StandardScaler()
X_data = scaler.fit_transform(X_data)  # Ahora X_data tiene media≈0 y std≈1 por columna
X_data

### Include $x_0$ for the bias term.
We want to include the *bias* term in the model, so we include an additional column (variable $x_0$) full of ones.

In [ ]:
# Concatenar columna de unos al inicio de la matriz de features
# X resultante tiene forma (n_samples x 3): [1, size_std, rooms_std]
X = np.c_[np.ones(X_data.shape[0]), X_data]
X

Here we also transform the target variable $y$ to thousand of dollars. This is also not required, but done here for convenience.

In [ ]:
# Convertir precios a miles de dólares para trabajar con números más manejables
# Esto no afecta el modelo, solo facilita la interpretación de theta
y = y_data / 1000
y

### Plot the data

Now that we have our data prepared, let's plot the independent variables (features) along with the dependent variable (target).

In [ ]:
# Crear figura
plt.figure(figsize=(8, 5))

# Scatter plot de size (estandarizado) vs precio
plt.scatter(X[:, 1], y, color='steelblue', alpha=0.7)

# Etiquetas y título
plt.xlabel("$Size$ (estandarizado)", fontsize=14)
plt.ylabel("$Price$ (miles USD)", fontsize=14)
plt.title("Tamaño de la casa vs Precio")

# Cuadrícula, ajuste de márgenes y mostrar
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# Crear figura
plt.figure(figsize=(8, 5))

# Scatter plot de rooms (estandarizado) vs precio
plt.scatter(X[:, 2], y, color='tomato', alpha=0.7)

# Etiquetas y título
plt.xlabel("$Rooms$ (estandarizado)", fontsize=14)
plt.ylabel("$Price$ (miles USD)", fontsize=14)
plt.title("Número de cuartos vs Precio")

# Cuadrícula, ajuste de márgenes y mostrar
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# Crear figura 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Scatter 3D: size (eje X), rooms (eje Y), precio (eje Z)
ax.scatter(X[:, 1], X[:, 2], y, c='steelblue', s=40, alpha=0.8)

# Etiquetas de cada eje y título
ax.set_xlabel('Size (std)')
ax.set_ylabel('Rooms (std)')
ax.set_zlabel('Price (k$)')
ax.set_title('Distribución 3D de los datos')

# Ángulo de cámara: elevación=20°, azimut=245°
ax.view_init(20, 245)

# Ajuste de márgenes y mostrar
plt.tight_layout()
plt.show()

### 1. Scikit-learn

In [ ]:
lr = LinearRegression(fit_intercept=False)
lr.fit(X, y)
print(lr.intercept_, lr.coef_)

$\hat{y} = 340.4127 + 109.4478 x_1 - 6.5784 x_2$

In [ ]:
lr.predict(X)

### Plot the data

In [ ]:
theta_hat = lr.coef_.T
theta_hat

In [ ]:
size_points, rooms_points = np.meshgrid([np.min(X[:,1]), np.max(X[:,1])],[np.min(X[:,2]), np.max(X[:,2])])
size_points, rooms_points

In [19]:
price_points = theta_hat[0] + theta_hat[1] * size_points + theta_hat[2] * rooms_points

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[:,1], X[:,2], y, c='r')
ax.set_xlabel('Size')
ax.set_ylabel('Rooms')
ax.set_zlabel('Price')
ax.plot_surface(size_points, rooms_points, price_points, alpha=0.2)
ax.view_init(10, 250)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig_layout = go.Layout(title='Fitted model', autosize=True, width=800, height=800,
                       scene = dict(xaxis=dict(title='Size'),
                                    yaxis=dict(title='Rooms'),
                                    zaxis=dict(title='Price')))

fig = px.scatter_3d(x=X[:,1].ravel(), y=X[:,2].ravel(), z=y.ravel())
fig.update_traces(marker=dict(size=3))
fig.update_layout(fig_layout)
fig.add_traces(go.Surface(x=size_points, y=rooms_points, z=price_points, opacity=0.2))

fig.show()

### 2. Normal equation

In [ ]:
# Your code

### 3. SVD pseudoinverse

In [ ]:
# Your code

### 4. Gradient descent (batch)

In [ ]:
# Your code

### 5. Gradient descent (stochastic)

In [ ]:
# Your code

### 6. Gradient descent (mini-batch)

Explicación: 

In [ ]:
def minibatch_gradient_descent(X, y, alpha=0.01, n_iter=5000, batch_size=8, random_state=42):
    """
    Implementa Mini-Batch Gradient Descent para regresión lineal.
    En cada iteración usa un subconjunto de tamaño batch_size.
    
    Args:
        X            : Matriz de features con bias (n x p)
        y            : Vector objetivo (n x 1)
        alpha        : Tasa de aprendizaje
        n_iter       : Número de iteraciones
        batch_size   : Tamaño del mini-batch
        random_state : Semilla para reproducibilidad
    Returns:
        theta        : Parámetros encontrados (p x 1)
        history      : Historial del costo J
    """
    np.random.seed(random_state)            # Fijar semilla para reproducibilidad
    m, p = X.shape                          # m = muestras, p = parámetros
    theta = np.zeros((p, 1))                # Inicializar θ en ceros
    history = []                            # Historial del costo global

    for i in range(n_iter):
        # Seleccionar aleatoriamente 'batch_size' índices del dataset
        indices = np.random.choice(m, size=batch_size, replace=False)  # Sin reemplazo

        # Extraer el mini-batch de features y etiquetas
        X_batch = X[indices, :]             # Sub-matriz de features (batch_size x p)
        y_batch = y[indices]                # Sub-vector objetivo (batch_size x 1)

        # Calcular el gradiente promediado sobre el mini-batch
        residuals = X_batch @ theta - y_batch           # Residuales del mini-batch
        gradient  = (1 / batch_size) * X_batch.T @ residuals  # Gradiente (p x 1)

        theta = theta - alpha * gradient    # Actualización de θ

        # Registrar el costo global para seguimiento
        history.append(cost_function(X, y, theta))

    return theta, history


# Ejecutar Mini-Batch GD con batch_size=8 (aprox. 1/5 del dataset)
theta_mb, history_mb = minibatch_gradient_descent(X, y, alpha=0.01, n_iter=5000, batch_size=8)

print("Parámetros (Mini-Batch GD, batch=8):")
print(f"  θ₀ (bias):  {theta_mb[0][0]:.4f}")
print(f"  θ₁ (size):  {theta_mb[1][0]:.4f}")
print(f"  θ₂ (rooms): {theta_mb[2][0]:.4f}")

mse_mb = mean_squared_error(y, X @ theta_mb)
print(f"MSE (Mini-Batch GD): {mse_mb:.4f}")

---
## 7. Comparación de los tres métodos de descenso por gradiente (curvas de costo)

In [ ]:
''' Comparar visualmente las tres curvas de convergencia en la misma gráfica
plt.figure(figsize=(11, 6))

# Cada variante con color e intensidad distintos para diferenciarlas
plt.plot(history_batch, label='Batch GD',     color='steelblue', linewidth=2)
plt.plot(history_mb,    label='Mini-Batch GD (b=8)', color='seagreen',  linewidth=1.5, alpha=0.9)
plt.plot(history_sgd,   label='SGD',          color='tomato',    linewidth=1,   alpha=0.6)

plt.xlabel("Iteración", fontsize=13)
plt.ylabel("Costo $J(\\theta)$", fontsize=13)
plt.title("Comparación de convergencia — Batch vs Mini-Batch vs SGD")
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.4)
plt.tight_layout()
plt.show() '''

---
## 8. Comparación de Parámetros obtenidos con todos los métodos

In [ ]:
'''# Construir un DataFrame comparativo con θ₀, θ₁, θ₂ y el MSE de cada método
results = pd.DataFrame({
    'Método': [
        'Scikit-learn',
        'Ecuación Normal',
        'SVD Pseudoinversa',
        'Batch GD',
        'SGD',
        'Mini-Batch GD'
    ],
    'θ₀ (bias)': [
        theta_sklearn[0][0],
        theta_normal[0][0],
        theta_pinv[0][0],
        theta_batch[0][0],
        theta_sgd[0][0],
        theta_mb[0][0]
    ],
    'θ₁ (size)': [
        theta_sklearn[1][0],
        theta_normal[1][0],
        theta_pinv[1][0],
        theta_batch[1][0],
        theta_sgd[1][0],
        theta_mb[1][0]
    ],
    'θ₂ (rooms)': [
        theta_sklearn[2][0],
        theta_normal[2][0],
        theta_pinv[2][0],
        theta_batch[2][0],
        theta_sgd[2][0],
        theta_mb[2][0]
    ],
    'MSE': [
        mse_sklearn,
        mse_normal,
        mse_pinv,
        mse_batch,
        mse_sgd,
        mse_mb
    ]
})

# Formatear a 4 decimales para lectura limpia
results = results.set_index('Método').round(4)
results'''

---
## Conclusiones

### Sobre la normalización
- **Scikit-learn, Ecuación Normal y SVD:**
- **Batch GD, SGD y Mini-Batch GD:**
### Sobre los parámetros obtenidos

### Sobre la convergencia
- **Batch GD:**
- **Mini-Batch GD:**
- **SGD:**